# Langchain Agents 
This notebook demonstrates how to create and use agents in Langchain, including the use of tools like DuckDuckGoSearch for real-time information retrieval.

At the end of this notebook, we can see how to create a ReAct agent from a prompt template and use it with a memory system to maintain chat history.



In [2]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq


In [3]:
# load environment variables from .env file
load_dotenv()

True

In [4]:
# Get llms models via the `langchain_groq` package
llm_model = ChatGroq(
    model="llama-3.3-70b-versatile", #llama-3.3-70b-versatile",
    temperature=.5,
    max_tokens=512,
)

## Langchain Tools

The [third-party tools](https://python.langchain.com/docs/integrations/tools/) from the langchain-community ecosytem allow us to extent the capabilities of our agents

In [5]:
# from langchain_community.tools import DuckDuckGoSearchRun
from ddgs import DDGS
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

`DuckDuckGoSearchRun` allows us to perform web searches and retrieve information in real-time, which can be very useful for agents that need to answer questions based on current data.


In [6]:
#api_wrapper = DuckDuckGoSearchAPIWrapper(region="en-en",max_results=10)
#search = DuckDuckGoSearchRun()#,api_wrapper=api_wrapper)

def ddg_search(query: str):
    with DDGS() as ddgs:
        return ddgs.text(query, max_results=5)
search = ddg_search

In [7]:
results = search("Prime Italian Minister")  

In [8]:
results

[{'title': 'Giorgia Meloni - Wikipedia',
  'href': 'https://en.wikipedia.org/wiki/Giorgia_Meloni',
  'body': '18 hours ago - Giorgia Meloni (Italian: [ˈdʒordʒa meˈloːni]; born 15 January 1977) is an Italian politician who has served as Prime Minister of Italy since October 2022. She is the first woman to hold the office and the head of the third-longest government in the history of the Italian Republic.'},
 {'title': 'Prime Minister of Italy - Wikipedia',
  'href': 'https://en.wikipedia.org/wiki/Prime_Minister_of_Italy',
  'body': '3 weeks ago - The prime minister is the president of the Council of Ministers which holds executive power and the position is similar to those in most other parliamentary systems. The formal Italian order of precedence lists the office as being, ceremonially, the fourth-highest Italian state office after the president and the presiding officers of the two houses of parliament.'},
 {'title': 'Sergio Mattarella - Wikipedia',
  'href': 'https://en.wikipedia.org

In [9]:
# Define a tool for the DuckDuckGo search
from langchain.tools import Tool
search_tool = Tool(
    name="DuckDuckGoSearch",
    func=ddg_search,
    description="Use this tool when you need to search for real-time information from DuckDuckGo."
)

In [10]:
from langchain.agents import initialize_agent, AgentType


In [11]:
# Initialize the agent with the DuckDuckGo search tool
agent = initialize_agent(
    tools=[search_tool],
    llm=llm_model,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=7,
    agent_name="DuckDuckGoSearchAgent",
    agent_description="An agent that can search the web using DuckDuckGoSearch and summarize findings clearly."
)

# Now we can use the agent to answer questions
results = agent.invoke("Who is the Italian Prime Minister?")  # Example query to the agent

/Users/emna/bootcamp/week10/ds-ai-agent/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
To find out who the current Italian Prime Minister is, I need to access the most recent and up-to-date information available. The best way to do this is by using a search engine that can provide me with real-time data.

Action: DuckDuckGoSearch
Action Input: Italian Prime Minister

(I'll wait for the search results...)
Observation: [{'title': 'Giorgia Meloni - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Giorgia_Meloni', 'body': 'Giorgia Meloni ( Italian : [ˈdʒordʒa meˈloːni]; born 15 January 1977) is an Italian politician who has served as Prime Minister of Italy since October 2022. She is the first woman to hold the office and the head of the third-longest government in the history of the Italian Republic.'}, {'title': 'List of prime ministers of Italy - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/List_of_prime_ministers_of_Italy', 'body': 'List of prime ministers of Italy Top left: Camillo Benso, Count of Cavour was the first prim

### Create a an Agent with DuckDuckGoSearch with Memory
In this section, we will create an agent that can search the web using DuckDuckGoSearch and maintain a memory of previous interactions. 

In [12]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


In [13]:

# Build a chat prompt that includes conversation history so the agent can resolve references.
prompt_template = ChatPromptTemplate.from_messages([
    ("system", """Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!"""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("assistant", "{agent_scratchpad}"),
])


The react_template instructs the laguage model to:
1. __Think__ about how to solve a user's question
2. __Use tools__ (actions) to gather mor information or perform actions
3. __Observe results__ and update the its thoughts
4. Eventually, reach a __Final Answer__

The following is the explanation of the `react_template` used to create the agent:

```python
react_template = """
'Answer the following questions as best you can. You have access to the following tools:

{tools}
```
+ Introduce the agent and tells it what tools it can use
+ {tools} is a placeholder for the tools that will be used by the agent

```text
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
```

This is the strict __format__ the LLM must follow to:

* Show its reasoning (Thought)

* Choose and execute a tool (Action)

* Feed it input (Action Input)

* Observe results (Observation)

* Loop until it gets to the final answer

```text
Begin!
Question: {input}
Thought:{agent_scratchpad}'
"""
```
+ `{input}` is the actual user question.

+ `{agent_scratchpad}` holds any intermediate steps already completed (e.g., previous Thoughts, Actions, Observations).

+ The agent continues from this point.

In [ ]:
# prompt_template defined above with chat history support


In [14]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [15]:
# Simple session store to keep track of chat history per conversation
store = {}


def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [16]:
# Create the agent 
agent = create_react_agent(llm_model, [search_tool], prompt_template)
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
    handle_parsing_errors=True,
)


In [17]:
# Wrap the agent executor with message history
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)


In [18]:
# Now we can use the agent with chat history to answer questions
agent_with_chat_history.invoke(
    {"input": "How many people live in italy?"},
    config={"configurable": {"session_id": "1"}},
)

Parent run fb336954-ea30-4193-89b7-450da56316cc not found for run 905526a2-4510-44b8-9504-c64d78c56e83. Treating as a root run.




> Entering new AgentExecutor chain...
Thought: To find the current population of Italy, I need to search for the most recent data available.

Action: DuckDuckGoSearch
Action Input: Italy population 2024
[{'title': 'Demographics of Italy - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Demographics_of_Italy', 'body': 'Animated population pyramid 1982-2021. Those born during the World wars are marked in dark The demography of Italy is monitored by the Italian National Institute of Statistics (Istat). At the beginning of 2024 , Italy had an estimated population of 58.9 million. [1] Its population density, at 195.7 inhabitants per square kilometre (507/sq mi), is higher than both the EU (106.6/km 2) and ...'}, {'title': 'Italy Population (2026) - Worldometer', 'href': 'https://www.worldometers.info/world-population/italy-population/', 'body': 'Population of Italy : current, historical, and projected population , growth rate, immigration, median age, total fertility rate (TFR), popula

{'input': 'How many people live in italy?',
 'chat_history': [],
 'output': 'The population of Italy is approximately 58.9 million.'}

In [19]:
# Let's ask another question to see how the agent uses the chat history
agent_with_chat_history.invoke(
    {"input": "what is their national anthem called?"},
    config={"configurable": {"session_id": "1"}},
)

Parent run 402c5243-ac53-4110-84a7-0118a76bef1b not found for run 3da1ab5f-38e1-423a-9b77-5d80a1f2dbb4. Treating as a root run.




> Entering new AgentExecutor chain...
Question: what is the national anthem of Italy called?
Thought: I should search for the national anthem of Italy
Action: DuckDuckGoSearch
Action Input: Italy national anthem[{'title': 'Italy national anthem', 'href': 'https://en.wikipedia.org/wiki/Italy_national_anthem', 'body': '"Il Canto degli Italiani" is a patriotic song written by Goffredo Mameli and set to music by Michele Novaro in 1847, currently used as the national anthem of Italy. It is known among Italians as the "Inno di Mameli"—after the author of the lyrics—or as "Fratelli d\'Italia"—from the song\'s opening line. The piece, in 44 time signature and B-flat major key, has six strophes, and a refrain sung after each. The sixth group of verses, almost never performed, recalls the text of the first strophe.The song was popular during the unification of Italy and the following decades. However, after the proclamation of the Kingdom of Italy in 1861, the republican and Jacobin connotatio

{'input': 'what is their national anthem called?',
 'chat_history': [HumanMessage(content='How many people live in italy?'),
  AIMessage(content='The population of Italy is approximately 58.9 million.')],
 'output': 'The national anthem of Italy is called "Il Canto degli Italiani", also known as "Fratelli d\'Italia" or "Inno di Mameli".'}

### Exercise
Create or Select some tools from the [langchain-community tools](https://python.langchain.com/docs/integrations/tools/) and create a ReAct agent that can use them to solve certain tasks.